In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Make sure the output directory exists
os.makedirs('../assets', exist_ok=True)

# Load data from CSV files
socioeconomic = pd.read_csv('../data/socioeconomic.csv')
consumption = pd.read_csv('../data/Consumption.csv')
regions = pd.read_csv('../data/Region.csv')

# Define translations for Danish to English
socio_translations = {
    "Gennemsnitshusstand": "Average Household",
    "Selvstændig": "Self-employed",
    "Lønmodtager på højeste niveau": "High Income",
    "Lønmodtager på mellemniveau": "Medium Income",
    "Lønmodtager på grundniveau": "Basic Income",
    "Arbejdsløs": "Unemployed", 
    "Uddannelsessøgende": "Student",
    "Pensionist, efterlønsmodtager": "Pensioner",
    "Ude af erhverv i øvrigt": "Not in Workforce"
}

# Clean and process socioeconomic data
socio_df = socioeconomic.copy()
socio_df.columns = [col.strip() for col in socio_df.columns]
socio_df = socio_df.rename(columns={
    'Socioøkonomisk status': 'Group',
    '09.8 Pakkerejser': 'Packages',
    '11.1 Restaurationstjenester': 'Restaurants',
    '11.2 Overnatningsfaciliteter': 'Accommodation'
})
socio_df['Group_EN'] = socio_df['Group'].map(socio_translations)
socio_df['Total'] = socio_df['Packages'] + socio_df['Restaurants'] + socio_df['Accommodation']

# Process consumption/age data
age_df = consumption.copy()
# Identify the fixed prices columns
package_col = [col for col in age_df.columns if 'Package Holidays - Fixed Prices' in col][0]
restaurant_col = [col for col in age_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
accommodation_col = [col for col in age_df.columns if 'Accommodation Services - Fixed Prices' in col][0]

age_df = age_df.rename(columns={
    package_col: 'Packages',
    restaurant_col: 'Restaurants',
    accommodation_col: 'Accommodation'
})
age_df['Total'] = age_df['Packages'] + age_df['Restaurants'] + age_df['Accommodation']

# Process region data
region_df = regions.copy()
# Identify the fixed prices columns
package_col = [col for col in region_df.columns if 'Package Holidays - Fixed Prices' in col][0]
restaurant_col = [col for col in region_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
accommodation_col = [col for col in region_df.columns if 'Accommodation Services - Fixed Prices' in col][0]

region_df = region_df.rename(columns={
    package_col: 'Packages',
    restaurant_col: 'Restaurants',
    accommodation_col: 'Accommodation'
})
region_df['Total'] = region_df['Packages'] + region_df['Restaurants'] + region_df['Accommodation']

# Function to create more polished radar chart that closely matches the example image
def create_polished_radar_chart(df, title, filename, group_col='Group', display_col=None):
    """
    Create a radar chart where axes are demographic groups and polygons are spending categories,
    styled to closely match the example image.
    
    Parameters:
    df: DataFrame with the data
    title: Title for the chart
    filename: Output filename
    group_col: Column containing group identifiers
    display_col: Column containing display names (if None, uses group_col)
    """
    if display_col is None:
        display_col = group_col
        
    # Prepare the data
    groups = df[display_col].tolist()
    n_groups = len(groups)
    
    # Calculate angles for each group (divide the plot / number of variables)
    angles = np.linspace(0, 2 * np.pi, n_groups, endpoint=False).tolist()
    # Complete the loop
    angles += angles[:1]
    
    # Normalize to a 0-100 scale using the same max value for all categories
    # This helps maintain the proper relative scale between categories
    category_max = max(
        df['Packages'].max(),
        df['Restaurants'].max(),
        df['Accommodation'].max()
    )
    
    # Get values for each category
    packages_values = df['Packages'].tolist()
    packages_values += packages_values[:1]  # Complete the loop
    packages_norm = [(val / category_max * 100) for val in packages_values]
    
    restaurants_values = df['Restaurants'].tolist()
    restaurants_values += restaurants_values[:1]  # Complete the loop
    restaurants_norm = [(val / category_max * 100) for val in restaurants_values]
    
    accommodation_values = df['Accommodation'].tolist()
    accommodation_values += accommodation_values[:1]  # Complete the loop
    accommodation_norm = [(val / category_max * 100) for val in accommodation_values]
    
    # Calculate the position of groups to ensure proper labeling
    groups += groups[:1]  # Complete the loop for the labels
    
    # Create the plot with a white background like in the example
    fig = plt.figure(figsize=(10, 10), facecolor='white')
    ax = fig.add_subplot(111, polar=True)
    
    # Define colors exactly matching the example image
    category_colors = {
        'Packages': '#8c68af',      # Moderate purple
        'Restaurants': '#78c17c',   # Moderate green
        'Accommodation': '#f5d98f'  # Light yellow/orange
    }
    
    # Define Danish labels for the legend
    danish_labels = {
        'Packages': 'Rejsepakker',
        'Restaurants': 'Restauranter',
        'Accommodation': 'Overnatning'
    }
    
    # Set up clean gridlines that match the example image
    # Make gridlines lighter and use n concentric circles
    ax.grid(True, color='#e0e0e0', alpha=0.95, linestyle='-', linewidth=0.5)
    
    # Remove default radial grid lines and add custom ones
    ax.set_yticks([25, 50, 75, 100])
    ax.set_yticklabels([])  # Hide labels
    
    # Change the color of the center and spine
    ax.spines['polar'].set_visible(False)
    
    # Plot each category as in the example image
    # Accommodation (yellow polygon)
    ax.fill(angles, accommodation_norm, color=category_colors['Accommodation'], alpha=0.3, edgecolor=category_colors['Accommodation'], linewidth=1.3)
    
    # Restaurants (green polygon)
    ax.fill(angles, restaurants_norm, color=category_colors['Restaurants'], alpha=0.3, edgecolor=category_colors['Restaurants'], linewidth=1.3)
    
    # Packages (purple polygon)
    ax.fill(angles, packages_norm, color=category_colors['Packages'], alpha=0.3, edgecolor=category_colors['Packages'], linewidth=1.3)
    
    # Set the labels for each demographic group
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(groups[:-1], color='#82817f', fontsize=9)
    
    # Set title (no title text in example image, but we'll add one subtly)
    plt.title(title, size=14, color='#707070', pad=15, fontweight='light')
    
    # Add a legend similar to the example
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', 
                 markerfacecolor=category_colors['Packages'], markersize=10, 
                 label=danish_labels['Packages']),
        plt.Line2D([0], [0], marker='o', color='w', 
                 markerfacecolor=category_colors['Restaurants'], markersize=10, 
                 label=danish_labels['Restaurants']),
        plt.Line2D([0], [0], marker='o', color='w', 
                 markerfacecolor=category_colors['Accommodation'], markersize=10, 
                 label=danish_labels['Accommodation'])
    ]
    
    ax.legend(handles=legend_elements, loc='lower center', 
            bbox_to_anchor=(0.5, -0.1), ncol=3, frameon=False, 
            handletextpad=0.5, fontsize=10)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(f'../assets/{filename}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Final polished radar visualization created and saved to '../assets/{filename}.png'")

# Create radar charts for each demographic type
# 1. Socioeconomic Groups
create_polished_radar_chart(
    socio_df,
    'Travel Spending Categories by Socioeconomic Group',
    'danish_travel_socioeconomic_final',
    'Group',
    'Group_EN'
)

# 2. Age Groups
create_polished_radar_chart(
    age_df,
    'Travel Spending Categories by Age Group',
    'danish_travel_age_final',
    'Age Group',
    'Age Group'
)

# 3. Regional Groups
create_polished_radar_chart(
    region_df,
    'Travel Spending Categories by Region',
    'danish_travel_region_final',
    'Region',
    'Region'
)

print("All final polished radar visualizations created successfully!")

Final polished radar visualization created and saved to '../assets/danish_travel_socioeconomic_final.png'
Final polished radar visualization created and saved to '../assets/danish_travel_age_final.png'
Final polished radar visualization created and saved to '../assets/danish_travel_region_final.png'
All final polished radar visualizations created successfully!


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Make sure the output directory exists
# Assuming you run this from a script in a 'scripts' folder, and 'data' and 'assets' are siblings
os.makedirs('../assets', exist_ok=True)
data_file_path = '../data/bubble_plot.csv' # Ensure this path is correct

# Load the data
try:
    df = pd.read_csv(data_file_path)
except FileNotFoundError:
    print(f"Error: The data file was not found at {data_file_path}")
    print("Please ensure the path to 'bubble_plot.csv' is correct.")
    exit()

# Clean the data
df['GDP'] = pd.to_numeric(df['GDP, PPP (current international $) [NY.GDP.MKTP.PP.CD]'].str.replace(',', ''), errors='coerce')
df['Population'] = pd.to_numeric(df['Population, total [SP.POP.TOTL]'], errors='coerce')
df['Departures'] = pd.to_numeric(df['International tourism, number of departures [ST.INT.DPRT]'].str.replace(',', ''), errors='coerce')
df['PerCapita'] = pd.to_numeric(df['International Tourism Departures per capita'], errors='coerce')

# Create continent mapping
continent_mapping = {
    'Denmark': 'Europe', 'Sweden': 'Europe', 'Norway': 'Europe', 'Finland': 'Europe', 'Iceland': 'Europe',
    'United Kingdom': 'Europe', 'Germany': 'Europe', 'France': 'Europe', 'Italy': 'Europe', 'Spain': 'Europe',
    'Netherlands': 'Europe', 'Belgium': 'Europe', 'Switzerland': 'Europe', 'Austria': 'Europe', 'Poland': 'Europe',
    'Czech Republic': 'Europe', 'Hungary': 'Europe', 'Romania': 'Europe', 'Portugal': 'Europe', 'Greece': 'Europe',
    'Ireland': 'Europe', 'Luxembourg': 'Europe', 'Slovenia': 'Europe', 'Croatia': 'Europe', 'Lithuania': 'Europe',
    'Latvia': 'Europe', 'Estonia': 'Europe', 'Slovakia': 'Europe', 'Bulgaria': 'Europe',
    'United States': 'North America', 'Canada': 'North America', 'Mexico': 'North America',
    'China': 'Asia', 'Japan': 'Asia', 'Korea, Rep.': 'Asia', 'India': 'Asia', 'Indonesia': 'Asia',
    'Singapore': 'Asia', 'Malaysia': 'Asia', 'Thailand': 'Asia', 'Vietnam': 'Asia', 'Philippines': 'Asia',
    'Hong Kong SAR, China': 'Asia', 'Russian Federation': 'Asia', 'Taiwan, China': 'Asia',
    'Australia': 'Oceania', 'New Zealand': 'Oceania',
    'South Africa': 'Africa', 'Egypt, Arab Rep.': 'Africa', 'Nigeria': 'Africa', 'Kenya': 'Africa', 'Morocco': 'Africa',
    'Brazil': 'South America', 'Argentina': 'South America', 'Chile': 'South America', 'Colombia': 'South America',
    'Peru': 'South America', 'Venezuela, RB': 'South America',
    'Saudi Arabia': 'Middle East', 'United Arab Emirates': 'Middle East', 'Turkey': 'Middle East', 'Israel': 'Middle East',
    'Albania': 'Europe', 'Belarus': 'Europe', 'Bosnia and Herzegovina': 'Europe', 'Kazakhstan': 'Asia',
    'Mongolia': 'Asia', 'North Korea': 'Asia', 'Sri Lanka': 'Asia', 'Pakistan': 'Asia', 'Bangladesh': 'Asia',
    'Myanmar': 'Asia', 'Nepal': 'Asia', 'Lebanon': 'Middle East', 'Jordan': 'Middle East', 'Kuwait': 'Middle East',
    'Bahrain': 'Middle East', 'Qatar': 'Middle East', 'Oman': 'Middle East', 'Yemen': 'Middle East',
    'Ethiopia': 'Africa', 'Ghana': 'Africa', 'Tanzania': 'Africa', 'Uganda': 'Africa', 'Zambia': 'Africa',
    'Zimbabwe': 'Africa', 'Cameroon': 'Africa', 'Senegal': 'Africa', 'Mali': 'Africa', 'Niger': 'Africa',
    'Chad': 'Africa', 'Sudan': 'Africa', 'Libya': 'Africa',
    'Tunisia': 'Africa', 'Algeria': 'Africa',
    'Ecuador': 'South America', 'Bolivia': 'South America', 'Paraguay': 'South America', 'Uruguay': 'South America',
    'Papua New Guinea': 'Oceania', 'Fiji': 'Oceania', 'Solomon Islands': 'Oceania'
}

df['Continent'] = df['Country Name'].map(continent_mapping).fillna('Other')
df_clean = df.dropna(subset=['GDP', 'PerCapita', 'Population', 'Continent']).copy()
years = sorted(df_clean['Time'].unique())
years = [year for year in years if 1996 <= year <= 2019]

continent_colors = {
    'Europe': '#374c80', 'North America': '#7a5195', 'Asia': '#bc5090',
    'Oceania': '#ef5675', 'Africa': '#ff764a', 'South America': '#ffa600',
    'Middle East': '#c1666b'
}
df_clean = df_clean[df_clean['Continent'] != 'Other'].copy()

if df_clean.empty or not years:
    print("Dataframe is empty after cleaning or no valid years found. Cannot proceed.")
    exit()

df_clean['GDP_Billion'] = df_clean['GDP'] / 1e9
df_clean['IsDenmark'] = df_clean['Country Name'] == 'Denmark'

fig = go.Figure()
current_trace_idx = 0
# Add legend traces
for continent in continent_colors.keys():
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', name=continent,
                             marker=dict(size=10, color=continent_colors[continent]),
                             showlegend=True, legendgroup=continent))
    current_trace_idx += 1
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', name='🇩🇰 Denmark',
                         marker=dict(size=15, color='#003f5c', line=dict(width=3, color='#C60C30'), symbol='star'),
                         showlegend=True, legendgroup='Denmark'))
current_trace_idx += 1

data_trace_indices = []
continent_order_for_traces = []
first_year = years[0]

# Add initial data traces for the first year
for continent in continent_colors.keys():
    data_trace_indices.append(current_trace_idx)
    continent_order_for_traces.append(continent)
    continent_data_initial = df_clean[(df_clean['Time'] == first_year) & (df_clean['Continent'] == continent) & (~df_clean['IsDenmark'])]
    fig.add_trace(go.Scatter(
        x=continent_data_initial['GDP_Billion'], y=continent_data_initial['PerCapita'],
        mode='markers', name=continent, text=continent_data_initial['Country Name'],
        hovertemplate='<b>%{text}</b><br>GDP: $%{x:.1f}B<br>Tourism/Capita: %{y:.3f}<br><extra></extra>',
        marker=dict(size=np.sqrt(continent_data_initial['Population'] / 2000000) * 10 if not continent_data_initial.empty else [],
                    sizemin=6, color=continent_colors[continent], opacity=0.7, line=dict(width=1, color='white')),
        visible=True, legendgroup=continent, showlegend=False))
    current_trace_idx += 1

data_trace_indices.append(current_trace_idx)
continent_order_for_traces.append('Denmark')
denmark_data_initial = df_clean[(df_clean['Time'] == first_year) & (df_clean['IsDenmark'])]
fig.add_trace(go.Scatter(
    x=denmark_data_initial['GDP_Billion'], y=denmark_data_initial['PerCapita'],
    mode='markers', name='Denmark', text=(['Denmark'] * len(denmark_data_initial)) if not denmark_data_initial.empty else [],
    hovertemplate='<b>Denmark</b><br>GDP: $%{x:.1f}B<br>Tourism/Capita: %{y:.3f}<br><extra></extra>',
    marker=dict(size=np.sqrt(denmark_data_initial['Population'] / 1500000) * 15 if not denmark_data_initial.empty else 15,
                color='#003f5c', line=dict(width=3, color='#C60C30'), symbol='star'),
    visible=True, legendgroup='Denmark', showlegend=False))

# --- Create Frames for Animation ---
plotly_frames = []
for year in years:
    frame_data_payload = []
    for continent_or_denmark in continent_order_for_traces:
        if continent_or_denmark == 'Denmark':
            current_year_data = df_clean[(df_clean['Time'] == year) & (df_clean['IsDenmark'])]
            marker_size = (np.sqrt(current_year_data['Population'] / 1500000) * 15).tolist() if not current_year_data.empty else []
            trace_update_dict = {'x': current_year_data['GDP_Billion'].tolist(),
                                 'y': current_year_data['PerCapita'].tolist(),
                                 'marker': {'size': marker_size},
                                 'text': (['Denmark'] * len(current_year_data)) if not current_year_data.empty else []}
        else:
            current_year_data = df_clean[(df_clean['Time'] == year) & (df_clean['Continent'] == continent_or_denmark) & (~df_clean['IsDenmark'])]
            marker_size = (np.sqrt(current_year_data['Population'] / 2000000) * 10).tolist() if not current_year_data.empty else []
            trace_update_dict = {'x': current_year_data['GDP_Billion'].tolist(),
                                 'y': current_year_data['PerCapita'].tolist(),
                                 'marker': {'size': marker_size},
                                 'text': current_year_data['Country Name'].tolist()}
        frame_data_payload.append(trace_update_dict)
    plotly_frames.append(go.Frame(data=frame_data_payload, name=str(year), traces=data_trace_indices))
fig.frames = plotly_frames

# Create slider steps - MODIFIED TO USE METHOD "ANIMATE"
steps = []
for year in years:
    step = dict(
        method="animate", # Changed from "restyle"
        args=[[str(year)],  # Target frame name to animate to
              {"frame": {"duration": 0, "redraw": True}, # duration 0 for immediate jump when slider is moved
               "mode": "immediate",
               "transition": {"duration": 0}} # No transition for manual slider interaction
             ],
        label=str(year) # Label of the slider step, matches frame name
    )
    steps.append(step)

# Update layout
fig.update_layout(
    title="Denmark's Tourism Journey (1996-2019) - Animated",
    xaxis=dict(title="GDP (billions USD PPP)", type='log',
               range=[np.log10(max(0.01, df_clean['GDP_Billion'].min(skipna=True) * 0.8 if pd.notna(df_clean['GDP_Billion'].min(skipna=True)) else 0.01)),
                      np.log10(max(1, df_clean['GDP_Billion'].max(skipna=True) * 1.2 if pd.notna(df_clean['GDP_Billion'].max(skipna=True)) else 1))],
               tickmode='array',
               tickvals=[0.1,0.2,0.5,1,2,5,10,20,50,100,200,500,1000,2000,5000,10000,20000],
               ticktext=['0.1B','0.2B','0.5B','1B','2B','5B','10B','20B','50B','100B','200B','500B','1T','2T','5T','10T','20T'],
               showgrid=False),
    yaxis=dict(title="International Tourism Departures per Capita",
               range=[-0.1, max(0.1, df_clean['PerCapita'].max(skipna=True) * 1.15 if pd.notna(df_clean['PerCapita'].max(skipna=True)) else 0.1)],
               showgrid=True, gridcolor='lightgray'),
    showlegend=True,
    legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1.02),
    plot_bgcolor='white', width=None, height=None, autosize=True,
    updatemenus=[{
        'type': 'buttons', 'showactive': False, 'y': -0.15, 'x': 0.5, 'xanchor': 'center',
        'yanchor': 'top', 'direction': 'left', 'pad': {'t': 10,'b':10, 'r': 10},
        'buttons': [
            {'label': '▶️ Play', 'method': 'animate',
             'args': [None,  # Animate all frames by their names (None uses fig.frames)
                      {'frame': {'duration': 700, 'redraw': True},
                       'fromcurrent': True,
                       'transition': {'duration': 300, 'easing': 'quadratic-in-out'},
                       'mode': 'immediate'}]}, # mode 'immediate' ensures it starts on click
            {'label': '⏸️ Pause', 'method': 'animate',
             'args': [[None], # Passing [None] to animation name target list stops current animation
                      {'frame': {'duration': 0, 'redraw': False}, # duration 0
                       'mode': 'immediate', # ensures action is immediate
                       'transition': {'duration': 0}}]}
        ]
    }],
    sliders=[{
        'active': 0, # Initial active step index (first year)
        'currentvalue': {'prefix': 'Year: ', 'visible': True, 'xanchor': 'right'},
        'pad': {'t': 20, 'b': 40},
        'steps': steps # Uses the modified steps
    }]
)

output_filename = '../assets/denmark_tourism_bubble_final_animation.html'
fig.write_html(
    output_filename,
    include_plotlyjs='cdn',
    config={'responsive': True, 'displayModeBar': True, 'modeBarButtonsToRemove': ['lasso2d', 'select2d']}
)
print(f"Final animated plot saved: {output_filename}")

Countries not mapped to continents (will be excluded): ['Angola', 'Azerbaijan', 'Burundi', 'Cambodia', 'Costa Rica', 'Cyprus', 'Dominican Republic', 'El Salvador', 'Georgia', 'Guatemala', 'Maldives', 'Mauritius', 'Nicaragua', 'Panama', 'Puerto Rico', 'Seychelles', 'Trinidad and Tobago', 'Turkiye', 'Turkmenistan', 'Vanuatu', 'Sierra Leone', 'Uzbekistan', 'Congo, Dem. Rep.', 'Gabon', 'Palau', 'Tuvalu', 'Samoa', 'Tajikistan', 'Armenia', 'Bermuda', 'Central African Republic', 'Czechia', 'Eswatini', 'Gambia, The', 'Macao SAR, China', 'Slovak Republic', 'Sao Tome and Principe', 'Ukraine', 'Kyrgyz Republic', 'Malta', 'Moldova', 'Honduras', 'Lao PDR', 'Congo, Rep.', 'Iran, Islamic Rep.', 'Togo', 'Bhutan']
Interactive and animated bubble plot with slider synchronization created successfully!
File saved: ../assets/denmark_tourism_bubble_animated_slider_sync.html
